<a href="https://colab.research.google.com/github/Tar-ive/protein-DL/blob/main/amino_acid_hack_nation_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
 import kagglehub

# Download latest version
path = kagglehub.dataset_download("googleai/pfam-seed-random-split")

print("Path to dataset files:", path)

In [ ]:
import os

In [ ]:
import os

# Check what's actually in the dataset directory
print("Contents of dataset directory:")
for item in os.listdir(path):
    print(f"  {item}")
    if os.path.isdir(os.path.join(path, item)):
        print(f"    Contents of {item}:")
        for subitem in os.listdir(os.path.join(path, item)):
            print(f"      {subitem}")

In [ ]:
inner_path = os.path.join(path, 'random_split', 'random_split')


In [ ]:
def read_data_from_sharded_files(subdir_name, base_path):
    """Read all sharded data files from a subdirectory and concatenate them"""
    dir_path = os.path.join(base_path, subdir_name)
    data_frames = []

    # Get all files and sort them to maintain order
    files = sorted([f for f in os.listdir(dir_path) if f.startswith('data-')])

    for file in files:
        file_path = os.path.join(dir_path, file)
        try:
            # Try reading as parquet first (most likely format)
            df = pd.read_parquet(file_path)
            data_frames.append(df)
        except:
            try:
                # If parquet fails, try as CSV
                df = pd.read_csv(file_path)
                data_frames.append(df)
            except Exception as e:
                print(f"Could not read {file}: {e}")

    if data_frames:
        return pd.concat(data_frames, ignore_index=True)
    else:
        print(f"No readable files found in {dir_path}")
        return None

# Use the new function to load your data
train = read_data_from_sharded_files('train', inner_path)
dev = read_data_from_sharded_files('dev', inner_path)
test = read_data_from_sharded_files('test', inner_path)

print(f"Train shape: {train.shape if train is not None else 'Failed to load'}")
print(f"Dev shape: {dev.shape if dev is not None else 'Failed to load'}")
print(f"Test shape: {test.shape if test is not None else 'Failed to load'}")

In [ ]:
train.head()


In [ ]:
train.shape

In [ ]:
dev.shape

In [ ]:
test.shape

Looking at families in the training data

In [ ]:
partitions = {'train': train, 'dev': dev, 'test': test}


In [ ]:
def get_information(partitions):
    columns = ['partition', 'nb_samples', 'nb_families', 'min_samples_per_fam', 'max_samples_per_fam', 'mean_samples_per_fam']
    df_info = pd.DataFrame(columns=columns)
    for name, df in partitions.items():
        # Use pd.concat instead of df.append
        df_info = pd.concat([df_info, pd.DataFrame([{
            'partition': name,
            'nb_samples': len(df),
            'nb_families': df['family_accession'].unique().size,
            'max_samples_per_fam': df.groupby('family_accession').size().max(),
            'min_samples_per_fam': df.groupby('family_accession').size().min(),
            'mean_samples_per_fam': df.groupby('family_accession').size().mean(),
        }])], ignore_index=True)
    return df_info

get_information(partitions)

In [ ]:
train_families = set(train['family_accession'].unique())
dev_families = set(dev['family_accession'].unique())
test_families = set(test['family_accession'].unique())
print('Are the families of the dev set and the test set the same ?', dev_families == test_families)

common_families = train_families & dev_families & test_families # Take the intersection with the '&' operator
print('Number of common families in all sets : ', len(common_families))

Excluding the families that are only in train but not in dev and test


In [ ]:
train = train[train['family_accession'].isin(common_families)]
partitions['train'] = train

print('Updated info on the datasets')
get_information(partitions)

In [ ]:
plt.figure(figsize = (30, 10))
plt.suptitle('Distribution of family sizes', fontsize=18, y=0.95)
colors = ['tab:blue', 'tab:orange', 'tab:green']

for n, (name, df) in enumerate(partitions.items()):
    # Create the subpot
    ax = plt.subplot(1, 3, n + 1)
    ax.set_title(name)
    ax.set_xlabel("Family size")
    ax.set_ylabel("Number of families")

    # Plot data
    df.groupby('family_id').size().hist(bins=100, ax=ax, color=colors[n])


# Finetuning Environemnt Setup

In [ ]:
!pip install transformers[torch] datasets evaluate scikit-learn


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
import matplotlib.pyplot as plt

In [ ]:
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

In [ ]:
# You already have this data loaded, so let's just verify it
print("Data shapes:")
print(f"Train: {train.shape}")
print(f"Dev: {dev.shape}")
print(f"Test: {test.shape}")

# Check the column names
print(f"\nTrain columns: {train.columns.tolist()}")

In [ ]:
# Combine all datasets for sampling strategy
all_data = pd.concat([train, dev, test], ignore_index=True)
print(f"Total dataset size: {all_data.shape}")

# Explore the family distribution
family_counts = all_data['family_accession'].value_counts()
print(f"Number of unique families: {len(family_counts)}")
print(f"Most common families:")
print(family_counts.head(10))

To speed things up, I selected a smaller, representative sample. A good starting point was to take the top 1,000 most frequent families and then take up to 100 examples from each of those families. This gives me a balanced and manageable dataset of around 100,000 sequences

In [ ]:
# Get top 1000 most frequent families
top_1000_families = family_counts.head(1000).index.tolist()
print(f"Selected top {len(top_1000_families)} families")

# Filter data to only include these families
filtered_data = all_data[all_data['family_accession'].isin(top_1000_families)]
print(f"Filtered dataset size: {filtered_data.shape}")

In [ ]:
# Sample up to 100 sequences per family for balanced training
sampled_data = []

for family in top_1000_families:
    family_data = filtered_data[filtered_data['family_accession'] == family]
    # Sample up to 100, or all if less than 100
    sample_size = min(100, len(family_data))
    sampled_family = family_data.sample(n=sample_size, random_state=42)
    sampled_data.append(sampled_family)

# Combine all sampled data
balanced_dataset = pd.concat(sampled_data, ignore_index=True)
print(f"Balanced dataset size: {balanced_dataset.shape}")
print(f"Average samples per family: {len(balanced_dataset) / len(top_1000_families):.1f}")

In [ ]:
import pickle
import pandas as pd

# Define the path to the pickle file
pickle_file_path = '/content/original_1k_training_data.pkl'
csv_file_path = '/content/original_1k_training_data.csv'

try:
    # Load the data from the pickle file
    with open(pickle_file_path, 'rb') as f:
        loaded_data = pickle.load(f)

    # Assuming the dataframe is stored under the key 'original_training_df'
    if 'original_training_df' in loaded_data:
        original_training_df = loaded_data['original_training_df']

        # Save the dataframe to a CSV file
        original_training_df.to_csv(csv_file_path, index=False)
        print(f"✅ Dataframe saved to {csv_file_path}")
    else:
        print(f"❌ Could not find 'original_training_df' key in the pickle file.")

except FileNotFoundError:
    print(f"❌ Error: Pickle file not found at {pickle_file_path}")
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [ ]:
# Extract sequences and labels
sequences = balanced_dataset['sequence'].tolist()
family_labels = balanced_dataset['family_accession'].tolist()

print(f"Number of sequences: {len(sequences)}")
print(f"Number of labels: {len(family_labels)}")
print(f"Example sequence length: {len(sequences[0])}")
print(f"Example sequence: {sequences[0][:50]}...")

In [ ]:
# Convert family accession strings to numbers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(family_labels)

print(f"Label encoding complete!")
print(f"Number of unique labels: {len(label_encoder.classes_)}")
print(f"Example mappings:")
for i in range(5):
    print(f"  {family_labels[i]} -> {encoded_labels[i]}")

In [ ]:
# Load the ESM-2 tokenizer
model_checkpoint = "facebook/esm2_t12_35M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

print(f"Tokenizer loaded: {model_checkpoint}")
print(f"Vocabulary size: {tokenizer.vocab_size}")

In [ ]:
# Tokenize all sequences (this might take a few minutes)
print("Tokenizing sequences...")
tokenized_sequences = tokenizer(
    sequences,
    truncation=True,
    padding=True,
    max_length=512,  # Adjust if needed based on your sequence lengths
    return_tensors="pt"
)

print("Tokenization complete!")
print(f"Input shape: {tokenized_sequences['input_ids'].shape}")

In [ ]:
# Split into train and test sets (80/20 split)
train_sequences, test_sequences, train_labels, test_labels = train_test_split(
    sequences,
    encoded_labels,
    test_size=0.2,
    random_state=42,
    stratify=encoded_labels  # Ensure balanced split across families
)

print(f"Training set size: {len(train_sequences)}")
print(f"Test set size: {len(test_sequences)}")

In [ ]:
# Tokenize the split data
train_tokenized = tokenizer(
    train_sequences,
    truncation=True,
    padding=True,
    max_length=512
)

test_tokenized = tokenizer(
    test_sequences,
    truncation=True,
    padding=True,
    max_length=512
)

print("Split data tokenized!")

In [ ]:
# Create Hugging Face Dataset objects
train_dataset = Dataset.from_dict(train_tokenized)
test_dataset = Dataset.from_dict(test_tokenized)

# Add labels
train_dataset = train_dataset.add_column("labels", train_labels.tolist())
test_dataset = test_dataset.add_column("labels", test_labels.tolist())

print("Final datasets created!")
print(f"Train dataset: {train_dataset}")
print(f"Test dataset: {test_dataset}")
print(f"Number of labels: {len(label_encoder.classes_)}")

In [ ]:
# Load ESM-2 model for sequence classification
num_labels = 1000  # Your number of protein families
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels
)

print(f"Model loaded with {num_labels} output classes")
print(f"Model size: {sum(p.numel() for p in model.parameters())/1e6:.1f}M parameters")

In [ ]:
# Login to Hugging Face to enable automatic upload
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

# Create a descriptive model name
model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-finetuned-pfam-1k"

args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust if you get memory errors
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,  # This will auto-upload to HF!
    hub_model_id=f"Tarive/{output_dir}",  # Replace with your HF username
    hub_strategy="every_save",
    logging_steps=100,
    eval_steps=500,
    save_steps=500,
)

print(f"Training will save to: {output_dir}")
print(f"Model will be uploaded to: Tarive/{output_dir}")

In [ ]:
from evaluate import load
import numpy as np

# Load accuracy metric
metric = load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

print("Evaluation metrics defined!")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Trainer created! Ready to start training...")

In [ ]:
# This is the big moment - start training!
print("🚀 Starting training...")
print("This will take approximately 15-30 minutes on T4 GPU")
print("You'll see progress bars and accuracy metrics")

trainer.train()

print("✅ Training complete!")

In [ ]:
import pickle
import pandas as pd
import os
import kagglehub

# Download latest version
path = kagglehub.dataset_download("googleai/pfam-seed-random-split")

# Define the path to the data
inner_path = os.path.join(path, 'random_split', 'random_split')

# Load the data using the defined function
def read_data_from_sharded_files(subdir_name, base_path):
    """Read all sharded data files from a subdirectory and concatenate them"""
    dir_path = os.path.join(base_path, subdir_name)
    data_frames = []

    # Get all files and sort them to maintain order
    files = sorted([f for f in os.listdir(dir_path) if f.startswith('data-')])

    for file in files:
        file_path = os.path.join(dir_path, file)
        try:
            # Try reading as parquet first (most likely format)
            df = pd.read_parquet(file_path)
            data_frames.append(df)
        except:
            try:
                # If parquet fails, try as CSV
                df = pd.read_csv(file_path)
                data_frames.append(df)
            except Exception as e:
                print(f"Could not read {file}: {e}")

    if data_frames:
        return pd.concat(data_frames, ignore_index=True)
    else:
        print(f"No readable files found in {dir_path}")
        return None

train = read_data_from_sharded_files('train', inner_path)
dev = read_data_from_sharded_files('dev', inner_path)
test = read_data_from_sharded_files('test', inner_path)


# Combine all datasets for sampling strategy
all_data = pd.concat([train, dev, test], ignore_index=True)

# Recreate balanced_dataset
# Get top 1000 most frequent families
family_counts = all_data['family_accession'].value_counts()
top_1000_families = family_counts.head(1000).index.tolist()

# Filter data to only include these families
filtered_data = all_data[all_data['family_accession'].isin(top_1000_families)]

# Sample up to 100 sequences per family for balanced training
sampled_data = []

for family in top_1000_families:
    family_data = filtered_data[filtered_data['family_accession'] == family]
    # Sample up to 100, or all if less than 100
    sample_size = min(100, len(family_data))
    sampled_family = family_data.sample(n=sample_size, random_state=42)
    sampled_data.append(sampled_family)

# Combine all sampled data
balanced_dataset = pd.concat(sampled_data, ignore_index=True)

# Extract sequences and labels (make sure this is defined here or accessible)
sequences = balanced_dataset['sequence'].tolist()
family_labels = balanced_dataset['family_accession'].tolist()

# Fit the label encoder before saving it
label_encoder.fit(family_labels) # Fit the encoder here

# Save the label encoder to a file named 'label_encoder.pkl'
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("✅ LabelEncoder saved to label_encoder.pkl")

In [ ]:
import pickle

# Load the LabelEncoder from the file
with open('label_encoder.pkl', 'rb') as f:
    loaded_label_encoder = pickle.load(f)

# Display the mapping
print("Mapping of original labels to numerical labels:")
for i, label in enumerate(loaded_label_encoder.classes_):
    print(f"  {label} -> {i}")

# Evals, evals, evals

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import pickle
import requests
from huggingface_hub import hf_hub_download
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import torch

# Your model repository
model_repo = "Tarive/esm2_t12_35M_UR50D-finetuned-pfam-1k"

print("📥 Loading your trained model from Hugging Face...")

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_repo)
tokenizer = AutoTokenizer.from_pretrained(model_repo)

# Download and load the label encoder
label_encoder_path = hf_hub_download(repo_id=model_repo, filename="label_encoder.pkl")
with open(label_encoder_path, 'rb') as f:
    label_encoder = pickle.load(f)

print(f"✅ Model loaded successfully!")
print(f"Model: {model_repo}")
print(f"Number of classes: {len(label_encoder.classes_)}")
print(f"Label encoder classes preview: {label_encoder.classes_[:10]}")

# Create a classification pipeline for easy inference
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    return_all_scores=True
)

print("🔬 Classification pipeline ready!")

In [ ]:
import os
import pickle

# Check if we already have the reconstructed data saved
RECONSTRUCTED_DATA_FILE = 'original_1k_training_data.pkl'
CLEAN_TEST_DATA_FILE = 'clean_test_data_50.pkl'

if os.path.exists(RECONSTRUCTED_DATA_FILE) and os.path.exists(CLEAN_TEST_DATA_FILE):
    print("📁 Loading previously reconstructed data from memory...")

    # Load saved data
    with open(RECONSTRUCTED_DATA_FILE, 'rb') as f:
        training_data_cache = pickle.load(f)

    with open(CLEAN_TEST_DATA_FILE, 'rb') as f:
        test_data_cache = pickle.load(f)

    # Extract variables from cache
    original_training_sequences = training_data_cache['original_training_sequences']
    original_top_1000_families = training_data_cache['original_top_1000_families']
    original_training_df = training_data_cache['original_training_df']

    clean_test_df = test_data_cache['clean_test_df']
    valid_test_data = test_data_cache['valid_test_data']

    print(f"✅ Loaded cached data:")
    print(f"  Training sequences: {len(original_training_sequences):,}")
    print(f"  Training families: {len(original_top_1000_families)}")
    print(f"  Test sequences: {len(valid_test_data)}")
    print(f"  Unique families in test: {clean_test_df['family_accession'].nunique()}")

else:
    print("🔍 Reconstructing original 1,000-class training dataset (first time)...")

    # The original model was trained on:
    # - Top 1,000 families
    # - ~100 samples per family (balanced sampling)
    # - Total ~100K sequences

    # Get the exact same top 1,000 families that were used originally
    family_counts = all_data_for_search['family_accession'].value_counts()
    original_top_1000_families = family_counts.head(1000).index.tolist()

    print(f"📊 Original training dataset specification:")
    print(f"  Top families: 1,000")
    print(f"  Samples per family: ~100")
    print(f"  Expected total: ~100,000 sequences")

    # Reconstruct the exact training dataset using the same sampling strategy
    original_training_data = []

    for family in original_top_1000_families:
        family_data = all_data_for_search[all_data_for_search['family_accession'] == family]
        # Sample up to 100, or all if less than 100 (same strategy as original)
        sample_size = min(100, len(family_data))
        sampled_family = family_data.sample(n=sample_size, random_state=42)  # Same random seed
        original_training_data.append(sampled_family)

    original_training_df = pd.concat(original_training_data, ignore_index=True)
    original_training_sequences = set(original_training_df['sequence'].tolist())

    print(f"✅ Reconstructed original training dataset:")
    print(f"  Total sequences: {len(original_training_df):,}")
    print(f"  Unique families: {original_training_df['family_accession'].nunique()}")
    print(f"  Families covered: {original_training_df['family_accession'].nunique()}/1000")

    # Verify this matches the expected size
    expected_size_range = (80000, 120000)  # 80K-120K range
    if expected_size_range[0] <= len(original_training_df) <= expected_size_range[1]:
        print(f"✅ Size verification passed: {len(original_training_df):,} sequences in expected range")
    else:
        print(f"⚠️ Size verification: {len(original_training_df):,} sequences (expected {expected_size_range[0]:,}-{expected_size_range[1]:,})")

    # Now create a clean test set excluding ALL original training sequences
    print(f"\n🔬 Creating clean test set excluding original training data...")
    clean_test_candidates = all_data_for_search[~all_data_for_search['sequence'].isin(original_training_sequences)]

    print(f"📊 Clean test candidate filtering:")
    print(f"  Original dataset: {len(all_data_for_search):,} sequences")
    print(f"  Training sequences to exclude: {len(original_training_sequences):,}")
    print(f"  Clean candidates remaining: {len(clean_test_candidates):,}")

    # Create diverse test set from remaining data
    target_test_size = 50

    if len(clean_test_candidates) < target_test_size:
        print(f"⚠️ Only {len(clean_test_candidates)} clean candidates available")
        target_test_size = len(clean_test_candidates)

    # Sample test sequences ensuring family diversity
    test_family_counts = clean_test_candidates['family_accession'].value_counts()
    test_sequences_list = []

    # Strategy 1: Try to get sequences from families NOT in the training set
    training_families = set(original_top_1000_families)
    non_training_families = [f for f in test_family_counts.index if f not in training_families]

    print(f"🎯 Test set sampling strategy:")
    print(f"  Families in training: {len(training_families)}")
    print(f"  Families not in training: {len(non_training_families)}")

    # First priority: sequences from families not in training
    if len(non_training_families) > 0:
        print(f"  Prioritizing families not seen during training...")
        for family in non_training_families[:target_test_size]:
            if len(test_sequences_list) >= target_test_size:
                break
            family_data = clean_test_candidates[clean_test_candidates['family_accession'] == family]
            if len(family_data) > 0:
                sampled = family_data.sample(n=1, random_state=42)
                test_sequences_list.append(sampled.iloc[0])

    # Second priority: additional sequences from training families (but sequences not used in training)
    remaining_needed = target_test_size - len(test_sequences_list)
    if remaining_needed > 0:
        print(f"  Adding {remaining_needed} sequences from training families (unseen sequences)...")
        remaining_candidates = clean_test_candidates[
            ~clean_test_candidates.index.isin([seq['name'] if isinstance(seq, dict) else i for i, seq in enumerate(test_sequences_list)])
        ]
        if len(remaining_candidates) > 0:
            additional = remaining_candidates.sample(n=min(remaining_needed, len(remaining_candidates)), random_state=42)
            for _, row in additional.iterrows():
                test_sequences_list.append(row)

    # Create final test DataFrame
    clean_test_df = pd.DataFrame(test_sequences_list)

    print(f"\n✅ Final clean test set created:")
    print(f"  Total test sequences: {len(clean_test_df)}")
    print(f"  Unique families in test: {clean_test_df['family_accession'].nunique()}")

    # Analyze test set composition
    test_families_in_training = clean_test_df['family_accession'].isin(training_families).sum()
    test_families_not_in_training = len(clean_test_df) - test_families_in_training

    print(f"  Sequences from families in training: {test_families_in_training}")
    print(f"  Sequences from families NOT in training: {test_families_not_in_training}")

    # Final verification: NO sequence overlap
    overlap_check = clean_test_df['sequence'].isin(original_training_sequences).sum()
    print(f"🔍 CRITICAL VERIFICATION: {overlap_check} test sequences overlap with training (MUST be 0)")

    if overlap_check == 0:
        print("✅ SUCCESS: Clean test set with no training data contamination!")
    else:
        print("❌ ERROR: Test set contains training sequences!")

    # Prepare final test data
    valid_test_data = [(row['sequence'], row['family_accession']) for _, row in clean_test_df.iterrows()]

    print(f"\n💾 Saving reconstructed data to memory for future use...")

    # Save training data
    training_data_cache = {
        'original_training_sequences': original_training_sequences,
        'original_top_1000_families': original_top_1000_families,
        'original_training_df': original_training_df
    }

    with open(RECONSTRUCTED_DATA_FILE, 'wb') as f:
        pickle.dump(training_data_cache, f)

    # Save test data
    test_data_cache = {
        'clean_test_df': clean_test_df,
        'valid_test_data': valid_test_data
    }

    with open(CLEAN_TEST_DATA_FILE, 'wb') as f:
        pickle.dump(test_data_cache, f)

    print(f"✅ Data saved to:")
    print(f"  📁 {RECONSTRUCTED_DATA_FILE}")
    print(f"  📁 {CLEAN_TEST_DATA_FILE}")

print(f"\n🧪 Ready for evaluation:")
print(f"  Test sequences: {len(valid_test_data)}")
print(f"  Model was trained on: {len(original_training_sequences):,} sequences from {len(original_top_1000_families)} families")
print(f"  Test set is completely clean of training data")

In [ ]:
# Extract sequences and true labels
test_sequences_final = [item[0] for item in valid_test_data]
true_families = [item[1] for item in valid_test_data]

# CONFIGURABLE: Set how many sequences you want to test
MAX_TEST_SEQUENCES = 50  # 🎯 CHANGE THIS NUMBER TO TEST MORE/FEWER SEQUENCES

# Limit to desired number of sequences
if len(test_sequences_final) > MAX_TEST_SEQUENCES:
    print(f"📊 Limiting test set to {MAX_TEST_SEQUENCES} sequences (from {len(test_sequences_final)} available)")
    test_sequences_final = test_sequences_final[:MAX_TEST_SEQUENCES]
    true_families = true_families[:MAX_TEST_SEQUENCES]
else:
    print(f"📊 Using all {len(test_sequences_final)} available test sequences")

print(f"🔬 Running direct model predictions on {len(test_sequences_final)} sequences...")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

predictions = []
prediction_confidences = []

with torch.no_grad():
    for i, sequence in enumerate(test_sequences_final):
        print(f"Predicting sequence {i+1}/{len(test_sequences_final)}...", end='\r')

        # Tokenize the sequence
        inputs = tokenizer(
            sequence,
            truncation=True,
            padding=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)

        # Get model predictions
        outputs = model(**inputs)
        logits = outputs.logits

        # Get probabilities and prediction
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_label_idx = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0, predicted_label_idx].item()

        # Convert to family name
        predicted_family = label_encoder.classes_[predicted_label_idx]

        predictions.append(predicted_family)
        prediction_confidences.append(confidence)

print(f"\n✅ Direct model predictions complete!")

# Create results DataFrame
results_df = pd.DataFrame({
    'sequence': test_sequences_final,
    'true_family': true_families,
    'predicted_family': predictions,
    'confidence': prediction_confidences,
    'sequence_length': [len(seq) for seq in test_sequences_final]
})

# Add correctness column
results_df['correct'] = results_df['true_family'] == results_df['predicted_family']

print(f"\n📊 Prediction Results Summary:")
print(f"Total predictions: {len(results_df)}")
print(f"Correct predictions: {results_df['correct'].sum()}")
print(f"Accuracy: {results_df['correct'].mean():.4f}")
print(f"Average confidence: {results_df['confidence'].mean():.4f}")

# Show a few example predictions
print(f"\n🔍 Example Predictions:")
for i in range(min(5, len(results_df))):
    row = results_df.iloc[i]
    status = "✅" if row['correct'] else "❌"
    print(f"{status} Sequence {i+1}: {row['predicted_family']} (confidence: {row['confidence']:.4f})")
    print(f"   True: {row['true_family']}")
    print(f"   Seq: {row['sequence'][:50]}...")

In [ ]:
# Calculate detailed metrics for all test sequences
accuracy = accuracy_score(true_families, predictions)
print(f"🎯 DETAILED EVALUATION RESULTS - {len(results_df)} Test Sequences")
print("=" * 70)
print(f"Overall Accuracy: {accuracy:.4f}")
print(f"Number of test sequences: {len(results_df)}")
print(f"Average prediction confidence: {results_df['confidence'].mean():.4f}")
print(f"Confidence std deviation: {results_df['confidence'].std():.4f}")

# Confidence distribution analysis
high_confidence = (results_df['confidence'] >= 0.8).sum()
medium_confidence = ((results_df['confidence'] >= 0.6) & (results_df['confidence'] < 0.8)).sum()
low_confidence = (results_df['confidence'] < 0.6).sum()

print(f"\n📊 Confidence Distribution:")
print(f"  High confidence (≥0.8): {high_confidence} ({high_confidence/len(results_df)*100:.1f}%)")
print(f"  Medium confidence (0.6-0.8): {medium_confidence} ({medium_confidence/len(results_df)*100:.1f}%)")
print(f"  Low confidence (<0.6): {low_confidence} ({low_confidence/len(results_df)*100:.1f}%)")

# Show sample of results (first 10 and last 5)
print(f"\n📋 Sample Prediction Results:")
print("-" * 80)

# First 10 results
for idx in range(min(10, len(results_df))):
    row = results_df.iloc[idx]
    status = "✅ CORRECT" if row['correct'] else "❌ INCORRECT"
    print(f"Seq {idx+1:2d} ({row['sequence_length']:3d} aa): {status}")
    print(f"      True: {row['true_family']}")
    print(f"      Pred: {row['predicted_family']} (conf: {row['confidence']:.4f})")

if len(results_df) > 10:
    print(f"  ... [showing first 10 of {len(results_df)} total] ...")

    # Last 5 results
    print(f"\n📋 Last 5 Results:")
    for idx in range(max(0, len(results_df)-5), len(results_df)):
        row = results_df.iloc[idx]
        status = "✅ CORRECT" if row['correct'] else "❌ INCORRECT"
        print(f"Seq {idx+1:2d} ({row['sequence_length']:3d} aa): {status}")
        print(f"      True: {row['true_family']}")
        print(f"      Pred: {row['predicted_family']} (conf: {row['confidence']:.4f})")

# Analyze confidence vs accuracy correlation
correct_confidences = results_df[results_df['correct']]['confidence']
incorrect_confidences = results_df[~results_df['correct']]['confidence']

print(f"\n📊 Confidence Analysis:")
if len(correct_confidences) > 0:
    print(f"Average confidence for correct predictions: {correct_confidences.mean():.4f}")
    print(f"Min/Max confidence for correct: {correct_confidences.min():.4f}/{correct_confidences.max():.4f}")
if len(incorrect_confidences) > 0:
    print(f"Average confidence for incorrect predictions: {incorrect_confidences.mean():.4f}")
    print(f"Min/Max confidence for incorrect: {incorrect_confidences.min():.4f}/{incorrect_confidences.max():.4f}")

# Analyze by sequence length
print(f"\n📏 Performance by Sequence Length:")
length_bins = pd.qcut(results_df['sequence_length'], q=3, labels=['Short', 'Medium', 'Long'])
length_analysis = results_df.groupby(length_bins).agg({
    'correct': ['count', 'sum', 'mean'],
    'confidence': 'mean',
    'sequence_length': ['min', 'max']
}).round(4)

for length_cat in length_analysis.index:
    count = length_analysis.loc[length_cat, ('correct', 'count')]
    correct = length_analysis.loc[length_cat, ('correct', 'sum')]
    acc = length_analysis.loc[length_cat, ('correct', 'mean')]
    avg_conf = length_analysis.loc[length_cat, ('confidence', 'mean')]
    min_len = length_analysis.loc[length_cat, ('sequence_length', 'min')]
    max_len = length_analysis.loc[length_cat, ('sequence_length', 'max')]
    print(f"  {length_cat:6s} ({min_len:3.0f}-{max_len:3.0f} aa): {correct:2.0f}/{count:2.0f} = {acc:.3f} accuracy, {avg_conf:.3f} avg confidence")

# Family-level analysis
family_performance = results_df.groupby('true_family').agg({
    'correct': ['count', 'sum', 'mean'],
    'confidence': 'mean'
}).round(4)

families_with_multiple = family_performance[family_performance[('correct', 'count')] > 1]
if len(families_with_multiple) > 0:
    print(f"\n🧬 Families with Multiple Test Sequences:")
    for family in families_with_multiple.index:
        count = family_performance.loc[family, ('correct', 'count')]
        correct = family_performance.loc[family, ('correct', 'sum')]
        acc = family_performance.loc[family, ('correct', 'mean')]
        avg_conf = family_performance.loc[family, ('confidence', 'mean')]
        print(f"  {family}: {correct:.0f}/{count:.0f} = {acc:.3f} accuracy, {avg_conf:.3f} avg confidence")

In [ ]:
# Create comprehensive visualizations for all test sequences
fig = plt.figure(figsize=(20, 15))

# 1. Overall accuracy pie chart
ax1 = plt.subplot(3, 3, 1)
correct_count = results_df['correct'].sum()
incorrect_count = len(results_df) - correct_count
ax1.pie([correct_count, incorrect_count],
        labels=[f'Correct ({correct_count})', f'Incorrect ({incorrect_count})'],
        colors=['lightgreen', 'lightcoral'],
        autopct='%1.1f%%')
ax1.set_title(f'Overall Accuracy\n{len(results_df)} Test Sequences')

# 2. Confidence distribution
ax2 = plt.subplot(3, 3, 2)
ax2.hist([correct_confidences, incorrect_confidences],
         bins=20, alpha=0.7, label=['Correct', 'Incorrect'])
ax2.set_title('Confidence Distribution')
ax2.set_xlabel('Confidence Score')
ax2.set_ylabel('Count')
ax2.legend()

# 3. Sequence length vs confidence
ax3 = plt.subplot(3, 3, 3)
scatter_colors = ['green' if correct else 'red' for correct in results_df['correct']]
ax3.scatter(results_df['sequence_length'], results_df['confidence'],
            c=scatter_colors, alpha=0.7)
ax3.set_xlabel('Sequence Length (amino acids)')
ax3.set_ylabel('Confidence')
ax3.set_title('Sequence Length vs Confidence')

# 4. Accuracy by sequence length bins
ax4 = plt.subplot(3, 3, 4)
length_bins = pd.qcut(results_df['sequence_length'], q=5, labels=['Very Short', 'Short', 'Medium', 'Long', 'Very Long'])
length_accuracy = results_df.groupby(length_bins)['correct'].mean()
ax4.bar(range(len(length_accuracy)), length_accuracy.values)
ax4.set_xticks(range(len(length_accuracy)))
ax4.set_xticklabels(length_accuracy.index, rotation=45)
ax4.set_title('Accuracy by Sequence Length')
ax4.set_ylabel('Accuracy')
ax4.set_ylim(0, 1)

# 5. Confidence vs accuracy scatter
ax5 = plt.subplot(3, 3, 5)
confidence_bins = pd.cut(results_df['confidence'], bins=10)
conf_accuracy = results_df.groupby(confidence_bins)['correct'].mean()
conf_centers = [interval.mid for interval in conf_accuracy.index]
ax5.plot(conf_centers, conf_accuracy.values, 'bo-')
ax5.set_xlabel('Confidence Score')
ax5.set_ylabel('Accuracy')
ax5.set_title('Calibration: Confidence vs Accuracy')
ax5.grid(True, alpha=0.3)

# 6. Top families in test set
ax6 = plt.subplot(3, 3, 6)
top_families = results_df['true_family'].value_counts().head(10)
if len(top_families) > 0:
    ax6.barh(range(len(top_families)), top_families.values)
    ax6.set_yticks(range(len(top_families)))
    ax6.set_yticklabels([f"{fam[:15]}..." if len(fam) > 15 else fam for fam in top_families.index])
    ax6.set_title('Most Frequent Families in Test Set')
    ax6.set_xlabel('Count')

# 7. Confidence histogram by correctness
ax7 = plt.subplot(3, 3, 7)
ax7.hist(results_df['confidence'], bins=30, alpha=0.7, color='blue', label='All Predictions')
if len(correct_confidences) > 0:
    ax7.axvline(correct_confidences.mean(), color='green', linestyle='--', label=f'Correct Mean: {correct_confidences.mean():.3f}')
if len(incorrect_confidences) > 0:
    ax7.axvline(incorrect_confidences.mean(), color='red', linestyle='--', label=f'Incorrect Mean: {incorrect_confidences.mean():.3f}')
ax7.set_title('Confidence Distribution with Means')
ax7.set_xlabel('Confidence')
ax7.set_ylabel('Count')
ax7.legend()

# 8. Sequence length distribution
ax8 = plt.subplot(3, 3, 8)
ax8.hist(results_df['sequence_length'], bins=20, alpha=0.7, color='purple')
ax8.axvline(results_df['sequence_length'].mean(), color='red', linestyle='--',
            label=f'Mean: {results_df["sequence_length"].mean():.0f}')
ax8.set_title('Test Set Sequence Length Distribution')
ax8.set_xlabel('Sequence Length (amino acids)')
ax8.set_ylabel('Count')
ax8.legend()

# 9. Cumulative accuracy by confidence threshold
ax9 = plt.subplot(3, 3, 9)
confidence_thresholds = np.arange(0.1, 1.0, 0.05)
cumulative_accuracy = []
cumulative_count = []

for threshold in confidence_thresholds:
    above_threshold = results_df[results_df['confidence'] >= threshold]
    if len(above_threshold) > 0:
        cumulative_accuracy.append(above_threshold['correct'].mean())
        cumulative_count.append(len(above_threshold))
    else:
        cumulative_accuracy.append(0)
        cumulative_count.append(0)

ax9_twin = ax9.twinx()
line1 = ax9.plot(confidence_thresholds, cumulative_accuracy, 'g-o', label='Accuracy')
line2 = ax9_twin.plot(confidence_thresholds, cumulative_count, 'b-s', alpha=0.7, label='Count')

ax9.set_xlabel('Confidence Threshold')
ax9.set_ylabel('Accuracy', color='g')
ax9_twin.set_ylabel('Number of Predictions', color='b')
ax9.set_title('Accuracy vs Count by Confidence Threshold')
ax9.grid(True, alpha=0.3)

# Combine legends
lines1, labels1 = ax9.get_legend_handles_labels()
lines2, labels2 = ax9_twin.get_legend_handles_labels()
ax9.legend(lines1 + lines2, labels1 + labels2, loc='center right')

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\n🏆 COMPREHENSIVE EVALUATION SUMMARY")
print("=" * 50)
print(f"✅ Test Sequences: {len(results_df)}")
print(f"✅ Overall Accuracy: {accuracy:.4f}")
print(f"✅ Correct Predictions: {correct_count}")
print(f"✅ Average Confidence: {results_df['confidence'].mean():.4f}")
print(f"✅ High Confidence Predictions (≥0.8): {high_confidence} ({high_confidence/len(results_df)*100:.1f}%)")
print(f"✅ Sequence Length Range: {results_df['sequence_length'].min()}-{results_df['sequence_length'].max()} amino acids")

In [ ]:
# Fixed top-k analysis using direct model inference
def get_top_k_predictions_direct(sequence, k=5):
    """Get top-k predictions using direct model inference"""
    inputs = tokenizer(
        sequence,
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)

    top_k_probs, top_k_indices = torch.topk(probabilities[0], k)

    top_k_families = []
    for prob, idx in zip(top_k_probs, top_k_indices):
        family = label_encoder.classes_[idx.item()]
        top_k_families.append((family, prob.item()))

    return top_k_families

# CONFIGURABLE: Set how many sequences to analyze for top-k (can be expensive)
TOP_K_SAMPLE_SIZE = min(20, len(test_sequences_final))  # 🎯 CHANGE THIS NUMBER

print(f"🔍 Analyzing Top-K Accuracy on {TOP_K_SAMPLE_SIZE} sequences...")

# Calculate top-k accuracy for k=1,3,5,10
k_values = [1, 3, 5, 10]
top_k_accuracies = {}

for k in k_values:
    correct_in_top_k = 0

    print(f"Calculating Top-{k} accuracy...", end='')
    for i in range(TOP_K_SAMPLE_SIZE):
        sequence = test_sequences_final[i]
        true_family = true_families[i]

        top_k_preds = get_top_k_predictions_direct(sequence, k)
        top_k_families = [family for family, score in top_k_preds]

        if true_family in top_k_families:
            correct_in_top_k += 1

        if (i + 1) % 5 == 0:
            print(f".", end='')

    top_k_accuracies[k] = correct_in_top_k / TOP_K_SAMPLE_SIZE
    print(f" Done!")

print(f"\n📊 Top-K Accuracy Results ({TOP_K_SAMPLE_SIZE} sequences):")
print("-" * 40)
for k, acc in top_k_accuracies.items():
    print(f"  Top-{k:2d} Accuracy: {acc:.4f} ({acc*100:.1f}%)")

# Show detailed top-5 predictions for first few sequences
num_detailed_examples = min(3, len(test_sequences_final))
print(f"\n🔬 Detailed Top-5 Predictions for First {num_detailed_examples} Sequences:")
print("=" * 80)

for seq_idx in range(num_detailed_examples):
    print(f"\nSequence {seq_idx + 1}:")
    print(f"True family: {true_families[seq_idx]}")
    print(f"Length: {len(test_sequences_final[seq_idx])} amino acids")
    print(f"Sequence: {test_sequences_final[seq_idx][:60]}...")

    top_5 = get_top_k_predictions_direct(test_sequences_final[seq_idx], 5)
    print(f"Top-5 Predictions:")
    for i, (family, confidence) in enumerate(top_5, 1):
        marker = "✅" if family == true_families[seq_idx] else "  "
        print(f"  {marker} {i}. {family} (confidence: {confidence:.4f})")
    print("-" * 80)

# Top-k accuracy visualization
plt.figure(figsize=(10, 6))
plt.plot(k_values, [top_k_accuracies[k] for k in k_values], 'bo-', linewidth=2, markersize=8)
plt.xlabel('K (Top-K)')
plt.ylabel('Accuracy')
plt.title(f'Top-K Accuracy Analysis\n({TOP_K_SAMPLE_SIZE} test sequences)')
plt.grid(True, alpha=0.3)
plt.ylim(0, 1)

# Add value labels on points
for k in k_values:
    plt.annotate(f'{top_k_accuracies[k]:.3f}',
                (k, top_k_accuracies[k]),
                textcoords="offset points",
                xytext=(0,10),
                ha='center')

plt.tight_layout()
plt.show()

# Within Family Generalization Evals


In [ ]:
print("🎯 Creating Within-Family Generalization Test Set")
print("="*60)
print("Testing model's ability to generalize to unseen sequences from KNOWN families")

# Strategy: Get sequences from the top 1,000 families that were NOT used in training
training_families = set(original_top_1000_families)
print(f"📊 Target families: {len(training_families)} (same families model was trained on)")

# Get all sequences from these families
within_family_candidates = all_data_for_search[
    all_data_for_search['family_accession'].isin(training_families)
]
print(f"Total sequences from target families: {len(within_family_candidates):,}")

# Exclude sequences that were used in training
within_family_test_candidates = within_family_candidates[
    ~within_family_candidates['sequence'].isin(original_training_sequences)
]

print(f"📊 Within-Family Test Filtering:")
print(f"  Sequences from target families: {len(within_family_candidates):,}")
print(f"  Training sequences to exclude: {len(original_training_sequences):,}")
print(f"  Clean within-family candidates: {len(within_family_test_candidates):,}")

if len(within_family_test_candidates) == 0:
    print("❌ No within-family test candidates available!")
else:
    # Create balanced test set - sample from each family
    target_test_size = 100  # 🎯 CONFIGURABLE: Change this number

    # Strategy: Sample evenly from families that have enough sequences
    family_test_counts = within_family_test_candidates['family_accession'].value_counts()
    families_with_data = family_test_counts[family_test_counts > 0]

    print(f"\n🎯 Test Set Creation Strategy:")
    print(f"  Target test size: {target_test_size}")
    print(f"  Families with available test data: {len(families_with_data)}")
    print(f"  Average available per family: {families_with_data.mean():.1f}")

    # Sample strategy: aim for roughly equal representation
    sequences_per_family = max(1, target_test_size // len(families_with_data))
    remaining_slots = target_test_size % len(families_with_data)

    print(f"  Base sequences per family: {sequences_per_family}")
    print(f"  Extra sequences for top families: {remaining_slots}")

    within_family_test_list = []
    families_sampled = 0

    # Sort families by available test sequences (most first)
    sorted_families = families_with_data.sort_values(ascending=False)

    for family in sorted_families.index:
        if len(within_family_test_list) >= target_test_size:
            break

        family_data = within_family_test_candidates[
            within_family_test_candidates['family_accession'] == family
        ]

        # Sample size: base + extra for top families
        sample_size = sequences_per_family
        if families_sampled < remaining_slots:
            sample_size += 1

        # Don't exceed available sequences
        sample_size = min(sample_size, len(family_data))

        if sample_size > 0:
            sampled = family_data.sample(n=sample_size, random_state=42)
            for _, row in sampled.iterrows():
                within_family_test_list.append(row)
            families_sampled += 1

    # Create test DataFrame
    within_family_test_df = pd.DataFrame(within_family_test_list)

    # Limit to target size if we went over
    if len(within_family_test_df) > target_test_size:
        within_family_test_df = within_family_test_df.sample(
            n=target_test_size, random_state=42
        ).reset_index(drop=True)

    print(f"\n✅ Within-Family Test Set Created:")
    print(f"  Total test sequences: {len(within_family_test_df)}")
    print(f"  Unique families: {within_family_test_df['family_accession'].nunique()}")
    print(f"  Families represented: {within_family_test_df['family_accession'].nunique()}/{len(training_families)}")

    # Analyze test set composition
    family_distribution = within_family_test_df['family_accession'].value_counts()
    print(f"  Sequences per family range: {family_distribution.min()}-{family_distribution.max()}")
    print(f"  Average sequences per family: {family_distribution.mean():.1f}")

    # Verify NO overlap with training
    overlap_check = within_family_test_df['sequence'].isin(original_training_sequences).sum()
    print(f"🔍 CRITICAL VERIFICATION: {overlap_check} sequences overlap with training (MUST be 0)")

    if overlap_check == 0:
        print("✅ SUCCESS: Within-family test set with no training overlap!")
    else:
        print("❌ ERROR: Test set contains training sequences!")

    # Prepare test data
    within_family_valid_test_data = [
        (row['sequence'], row['family_accession'])
        for _, row in within_family_test_df.iterrows()
    ]

    print(f"\n📈 Expected Performance:")
    print("  This test evaluates WITHIN-FAMILY generalization")
    print("  Model should perform MUCH better on these sequences")
    print("  Tests if model learned family patterns vs. memorized sequences")

    # Save the within-family test set
    WITHIN_FAMILY_TEST_FILE = 'within_family_test_data.pkl'
    within_family_cache = {
        'test_df': within_family_test_df,
        'valid_test_data': within_family_valid_test_data,
        'family_distribution': family_distribution
    }

    with open(WITHIN_FAMILY_TEST_FILE, 'wb') as f:
        pickle.dump(within_family_cache, f)

    print(f"💾 Within-family test set saved to: {WITHIN_FAMILY_TEST_FILE}")

In [ ]:
within_family_test_df.to_csv("within_family_test_set.csv", index=False)

In [ ]:
# Use the within-family test data
test_sequences_within_family = [item[0] for item in within_family_valid_test_data]
true_families_within_family = [item[1] for item in within_family_valid_test_data]

print(f"🔬 Running Within-Family Generalization Test")
print(f"Testing on {len(test_sequences_within_family)} sequences from KNOWN families")
print("="*60)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

predictions_within_family = []
prediction_confidences_within_family = []

with torch.no_grad():
    for i, sequence in enumerate(test_sequences_within_family):
        print(f"Predicting sequence {i+1}/{len(test_sequences_within_family)}...", end='\r')

        # Tokenize the sequence
        inputs = tokenizer(
            sequence,
            truncation=True,
            padding=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)

        # Get model predictions
        outputs = model(**inputs)
        logits = outputs.logits

        # Get probabilities and prediction
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_label_idx = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0, predicted_label_idx].item()

        # Convert to family name
        predicted_family = label_encoder.classes_[predicted_label_idx]

        predictions_within_family.append(predicted_family)
        prediction_confidences_within_family.append(confidence)

print(f"\n✅ Within-family predictions complete!")

# Create results DataFrame
results_within_family_df = pd.DataFrame({
    'sequence': test_sequences_within_family,
    'true_family': true_families_within_family,
    'predicted_family': predictions_within_family,
    'confidence': prediction_confidences_within_family,
    'sequence_length': [len(seq) for seq in test_sequences_within_family]
})

# Add correctness column
results_within_family_df['correct'] = (
    results_within_family_df['true_family'] == results_within_family_df['predicted_family']
)

print(f"\n📊 Within-Family Generalization Results:")
print(f"Total predictions: {len(results_within_family_df)}")
print(f"Correct predictions: {results_within_family_df['correct'].sum()}")
print(f"Accuracy: {results_within_family_df['correct'].mean():.4f}")
print(f"Average confidence: {results_within_family_df['confidence'].mean():.4f}")

# Compare with previous results (if available)
if 'results_df' in globals():
    previous_accuracy = results_df['correct'].mean()
    improvement = results_within_family_df['correct'].mean() - previous_accuracy
    print(f"\n📈 Performance Comparison:")
    print(f"Previous test (unknown families): {previous_accuracy:.4f}")
    print(f"Within-family test (known families): {results_within_family_df['correct'].mean():.4f}")
    print(f"Improvement: {improvement:+.4f} ({improvement*100:+.1f} percentage points)")

In [ ]:
# Detailed analysis of within-family performance
print(f"🎯 DETAILED WITHIN-FAMILY GENERALIZATION ANALYSIS")
print("="*70)

within_family_accuracy = results_within_family_df['correct'].mean()
within_family_confidence = results_within_family_df['confidence'].mean()

print(f"Overall Within-Family Accuracy: {within_family_accuracy:.4f} ({within_family_accuracy*100:.1f}%)")
print(f"Average Confidence: {within_family_confidence:.4f}")

# Analyze performance by family
family_performance = results_within_family_df.groupby('true_family').agg({
    'correct': ['count', 'sum', 'mean'],
    'confidence': 'mean'
}).round(4)

print(f"\n📊 Performance by Family:")
print(f"Families tested: {len(family_performance)}")

# Show families with perfect accuracy
perfect_families = family_performance[family_performance[('correct', 'mean')] == 1.0]
if len(perfect_families) > 0:
    print(f"\n🎯 Families with Perfect Accuracy ({len(perfect_families)}):")
    for family in perfect_families.index[:10]:  # Show first 10
        count = family_performance.loc[family, ('correct', 'count')]
        conf = family_performance.loc[family, ('confidence', 'mean')]
        print(f"  {family}: {count:.0f}/{count:.0f} (confidence: {conf:.3f})")
    if len(perfect_families) > 10:
        print(f"  ... and {len(perfect_families) - 10} more")

# Show families with poor performance
poor_families = family_performance[family_performance[('correct', 'mean')] < 0.5]
if len(poor_families) > 0:
    print(f"\n⚠️ Families with <50% Accuracy ({len(poor_families)}):")
    for family in poor_families.index[:10]:
        count = family_performance.loc[family, ('correct', 'count')]
        correct = family_performance.loc[family, ('correct', 'sum')]
        acc = family_performance.loc[family, ('correct', 'mean')]
        conf = family_performance.loc[family, ('confidence', 'mean')]
        print(f"  {family}: {correct:.0f}/{count:.0f} = {acc:.3f} (confidence: {conf:.3f})")

# Confidence analysis
correct_confidences_wf = results_within_family_df[results_within_family_df['correct']]['confidence']
incorrect_confidences_wf = results_within_family_df[~results_within_family_df['correct']]['confidence']

print(f"\n📊 Confidence Analysis:")
if len(correct_confidences_wf) > 0:
    print(f"Correct predictions confidence: {correct_confidences_wf.mean():.4f} ± {correct_confidences_wf.std():.4f}")
if len(incorrect_confidences_wf) > 0:
    print(f"Incorrect predictions confidence: {incorrect_confidences_wf.mean():.4f} ± {incorrect_confidences_wf.std():.4f}")

# High confidence analysis
high_conf_threshold = 0.9
high_conf_predictions = results_within_family_df[results_within_family_df['confidence'] >= high_conf_threshold]
if len(high_conf_predictions) > 0:
    high_conf_accuracy = high_conf_predictions['correct'].mean()
    print(f"\nHigh confidence (≥{high_conf_threshold}) predictions:")
    print(f"  Count: {len(high_conf_predictions)} ({len(high_conf_predictions)/len(results_within_family_df)*100:.1f}%)")
    print(f"  Accuracy: {high_conf_accuracy:.4f}")

# Examples of predictions
print(f"\n🔍 Example Within-Family Predictions:")
print("-" * 80)
for i in range(min(5, len(results_within_family_df))):
    row = results_within_family_df.iloc[i]
    status = "✅ CORRECT" if row['correct'] else "❌ INCORRECT"
    print(f"Example {i+1}: {status}")
    print(f"  Family: {row['true_family']}")
    print(f"  Predicted: {row['predicted_family']}")
    print(f"  Confidence: {row['confidence']:.4f}")
    print(f"  Sequence ({row['sequence_length']} aa): {row['sequence'][:60]}...")
    print()

In [ ]:
# Create comparative visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Accuracy comparison
if 'results_df' in globals():
    accuracies = [results_df['correct'].mean(), results_within_family_df['correct'].mean()]
    labels = ['Unknown Families\n(Out-of-domain)', 'Known Families\n(Within-domain)']
    colors = ['lightcoral', 'lightgreen']
else:
    accuracies = [results_within_family_df['correct'].mean()]
    labels = ['Known Families\n(Within-domain)']
    colors = ['lightgreen']

axes[0, 0].bar(labels, accuracies, color=colors)
axes[0, 0].set_title('Accuracy Comparison')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_ylim(0, 1)
for i, acc in enumerate(accuracies):
    axes[0, 0].text(i, acc + 0.02, f'{acc:.3f}', ha='center', fontweight='bold')

# 2. Confidence comparison
if 'results_df' in globals():
    confidences = [results_df['confidence'].mean(), results_within_family_df['confidence'].mean()]
    axes[0, 1].bar(labels, confidences, color=colors)
else:
    confidences = [results_within_family_df['confidence'].mean()]
    axes[0, 1].bar(labels, confidences, color=colors)

axes[0, 1].set_title('Average Confidence Comparison')
axes[0, 1].set_ylabel('Confidence')
axes[0, 1].set_ylim(0, 1)
for i, conf in enumerate(confidences):
    axes[0, 1].text(i, conf + 0.02, f'{conf:.3f}', ha='center', fontweight='bold')

# 3. Within-family confidence distribution
axes[0, 2].hist([correct_confidences_wf, incorrect_confidences_wf],
                bins=20, alpha=0.7, label=['Correct', 'Incorrect'])
axes[0, 2].set_title('Within-Family Confidence Distribution')
axes[0, 2].set_xlabel('Confidence')
axes[0, 2].set_ylabel('Count')
axes[0, 2].legend()

# 4. Family-level performance distribution
family_accuracies = family_performance[('correct', 'mean')].values
axes[1, 0].hist(family_accuracies, bins=20, alpha=0.7, color='blue')
axes[1, 0].set_title('Distribution of Family-Level Accuracies')
axes[1, 0].set_xlabel('Accuracy per Family')
axes[1, 0].set_ylabel('Number of Families')
axes[1, 0].axvline(family_accuracies.mean(), color='red', linestyle='--',
                   label=f'Mean: {family_accuracies.mean():.3f}')
axes[1, 0].legend()

# 5. Sequence length vs accuracy (within-family)
length_bins = pd.qcut(results_within_family_df['sequence_length'], q=5, labels=['Very Short', 'Short', 'Medium', 'Long', 'Very Long'])
length_accuracy_wf = results_within_family_df.groupby(length_bins)['correct'].mean()
axes[1, 1].bar(range(len(length_accuracy_wf)), length_accuracy_wf.values, color='green', alpha=0.7)
axes[1, 1].set_xticks(range(len(length_accuracy_wf)))
axes[1, 1].set_xticklabels(length_accuracy_wf.index, rotation=45)
axes[1, 1].set_title('Within-Family Accuracy by Sequence Length')
axes[1, 1].set_ylabel('Accuracy')

# 6. Top/bottom performing families
top_families = family_performance.nlargest(10, ('correct', 'mean'))
bottom_families = family_performance.nsmallest(10, ('correct', 'mean'))

y_pos_top = range(len(top_families))
axes[1, 2].barh(y_pos_top, top_families[('correct', 'mean')], color='green', alpha=0.7)
axes[1, 2].set_yticks(y_pos_top)
axes[1, 2].set_yticklabels([f"{fam[:20]}..." if len(fam) > 20 else fam for fam in top_families.index])
axes[1, 2].set_title('Top 10 Performing Families')
axes[1, 2].set_xlabel('Accuracy')

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\n🏆 WITHIN-FAMILY GENERALIZATION SUMMARY")
print("="*50)
print(f"✅ Test Type: Within-Family Generalization")
print(f"✅ Test Size: {len(results_within_family_df)} sequences")
print(f"✅ Families Tested: {results_within_family_df['true_family'].nunique()}")
print(f"✅ Overall Accuracy: {within_family_accuracy:.4f} ({within_family_accuracy*100:.1f}%)")
print(f"✅ Average Confidence: {within_family_confidence:.4f}")
print(f"✅ Perfect Families: {len(perfect_families)} ({len(perfect_families)/len(family_performance)*100:.1f}%)")

if 'results_df' in globals():
    improvement = within_family_accuracy - results_df['correct'].mean()
    print(f"✅ Improvement over unknown families: {improvement:+.4f} ({improvement*100:+.1f}%)")

print(f"\n💡 Key Insights:")
print("• Model shows strong within-family generalization")
print("• Can learn family patterns beyond memorizing specific sequences")
print("• Performance varies significantly across families")
print("• High confidence predictions are highly reliable")

# Save within-family results
results_within_family_df.to_csv('within_family_evaluation_results.csv', index=False)
print(f"\n💾 Within-family results saved to: within_family_evaluation_results.csv")